In [1]:
import pandas as pd
import folium
import json
import branca
import numpy as np
import requests
from folium import plugins
import missingno as msno
from branca.colormap import linear
import matplotlib as mpl
import matplotlib.pyplot as plt

In [24]:
store19 = pd.read_csv('store2019.csv',encoding='utf-8')
store20 = pd.read_csv('store2020.csv',encoding='utf-8')
store19


,adress,Latitude,Longitude,구,year,Q,store_type_code,store_type,adress_code,service_type_code,...,time6_count,male_count,female_count,age10_count,age20_count,age30_count,age40_count,age50_count,age60_count,store_count
0,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS300015,...,0.0,47.0,70.0,0.0,0.0,0.0,0.0,117.0,0.0,3.0
1,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100002,...,32.0,863.0,1254.0,174.0,815.0,471.0,424.0,179.0,53.0,1.0
2,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100005,...,1369.0,9568.0,12229.0,408.0,3952.0,7011.0,5095.0,3328.0,2002.0,5.0
3,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100004,...,100.0,1605.0,1839.0,31.0,1183.0,769.0,585.0,653.0,223.0,5.0
4,계동길,37.580942,126.986728,종로구,2019.0,1.0,A,골목상권,1000001.0,CS100010,...,338.0,13176.0,27821.0,677.0,13792.0,13387.0,6605.0,4474.0,2061.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135583,겸재로29길,37.590179,127.085804,중랑구,2019.0,4.0,A,골목상권,1000222.0,CS200037,...,23.0,59.0,7.0,0.0,7.0,22.0,22.0,15.0,0.0,3.0
135584,겸재로29길,37.590179,127.085804,중랑구,2019.0,4.0,A,골목상권,1000222.0,CS100002,...,22.0,3114.0,1670.0,57.0,458.0,919.0,1257.0,1215.0,879.0,1.0
135585,겸재로29길,37.590179,127.085804,중랑구,2019.0,4.0,A,골목상권,1000222.0,CS300004,...,0.0,632.0,1224.0,126.0,126.0,970.0,0.0,634.0,0.0,4.0
135586,겸재로29길,37.590179,127.085804,중랑구,2019.0,4.0,A,골목상권,1000222.0,CS200003,...,0.0,151.0,384.0,0.0,162.0,175.0,163.0,35.0,0.0,6.0


In [3]:
#지도데이터 불러오기
url = 'https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'
seoul_population = f'{url}/seoul_population.csv'

#알고싶은데이터는19년과 20년의 시간대별 매출양 차이
_19minor20=store19.iloc[:,68:74].groupby(store19['구']).sum() -\
store20.iloc[:,68:74].groupby(store19['구']).sum()

#인덱싱 재처리
_19minor20=_19minor20.reset_index()
_19minor20.mean()

time1_count    1313229.20
time2_count     -85091.24
time3_count      24198.68
time4_count     -78042.48
time5_count    2852563.96
time6_count    3027490.72
dtype: float64

In [4]:
# 데이터의 값을 측정해서 색 비율을 만들어준다.
# time1_count 값
colormap = linear.Blues_09.scale(
    _19minor20.time6_count.min(),
    _19minor20.time6_count.max()
)
colormap

In [5]:
#구별로 색을 입히자
_19minor20_dict = _19minor20.set_index('구')['time6_count'] #인덱스 
color_dict = {str(key): colormap(_19minor20_dict[key]) for key in _19minor20_dict.keys()}
color_dict

{'강남구': '#08326eff',
 '강동구': '#f5faffff',
 '강북구': '#ddeaf7ff',
 '강서구': '#d5e5f4ff',
 '관악구': '#4c99caff',
 '광진구': '#f3f9feff',
 '구로구': '#57a1cfff',
 '금천구': '#edf5fcff',
 '노원구': '#97c6e0ff',
 '도봉구': '#e3eef9ff',
 '동대문구': '#b3d3e9ff',
 '동작구': '#aacfe6ff',
 '마포구': '#08306bff',
 '서대문구': '#7ab6daff',
 '서초구': '#2778b9ff',
 '성동구': '#c4daefff',
 '성북구': '#aed1e7ff',
 '송파구': '#66abd5ff',
 '양천구': '#f7fbffff',
 '영등포구': '#99c7e0ff',
 '용산구': '#6fb0d7ff',
 '은평구': '#f0f7fdff',
 '종로구': '#4997c9ff',
 '중구': '#08468cff',
 '중랑구': '#0b559fff'}

In [31]:
# 지도영역 나누기
m = folium.Map([37.55784788416739, 127.0088620724302],
                   zoom_start = 10.5,
               tiles = ' cartodbpositron'
#원하는 타일골라쓰기
#Stamenwatercolor
#Stamentoner
#Stamenterrain
#cartodbpositron
#cartodbdark_matter
#       
)

folium.GeoJson(
    seoul_geo,
    name = '시간별거래량',                 #이름
    style_function=lambda feature: {
        'fillColor': color_dict[feature['properties']['name']], # 위에 만든 코드를 넣는곳
        'color': 'black',
        'weight' :1,                       #줄두께
        'dashArray':'8',                # 점선
        'fillOpacity' : 0.6,              #투명도
    }
).add_to(m)

colormap.caption = 'Population color scale'
colormap.add_to(m)

folium.LayerControl().add_to(m)

m

In [7]:
#검색데이터가 많아 연산이 많이 느려지는 것 같다.(중복도 제거해주자)
need=store20[['Latitude','Longitude','store_type']]
need['store_type'].value_counts()

골목상권    84892
발달상권    29533
전통시장    16773
관광특구     1123
Name: store_type, dtype: int64

In [8]:
# need.duplicated().value_counts() #중복이몇개인지확인
need=need.drop_duplicates()#중복값제거

In [9]:
need=need.reset_index()

In [10]:
need['store_type'].value_counts()

골목상권    1009
발달상권     206
전통시장     202
관광특구       6
Name: store_type, dtype: int64

In [26]:
m = folium.Map(
    location = ([37.55784788416739, 127.0088620724302]),
    tiles = 'cartodbpositron',zoom_start = 11
)

In [27]:
for i in range(len(need['Latitude'])):
    c = need['Latitude'][i],need['Longitude'][i]
    
    if need.loc[i, 'store_type'] == '골목상권': # 마크에 원하는 색상을 넣고싶을때
        
        icon_color = 'lightblue'
    elif need.loc[i, 'store_type'] == '발달상권':
        icon_color = 'darkblue'
    elif need.loc[i, 'store_type'] == '전통시장':
        icon_color = 'purple'
    elif need.loc[i, 'store_type'] == '관광특구':
        icon_color = 'yellow'
    folium.features.CircleMarker(
            location =[need['Latitude'][i],need['Longitude'][i]],
            radius = 3,
            popup=c,
            color = icon_color,
            fill=True,
            fill_color= 'white'
        ).add_to(m)
            
m

In [28]:
#지도데이터 불러오기
url = 'https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'
seoul_population = f'{url}/seoul_population.csv'

#상권별 점포수 19-20년 차이
StoreCount=store20.groupby(['store_type','구'])['store_count'].size() -\
store19.groupby(['store_type','구'])['store_count'].size()
StoreCount=StoreCount.unstack().T
StoreCount

store_type,골목상권,관광특구,발달상권,전통시장
구,,,,
강남구,-53.0,-5.0,-57.0,10.0
강동구,-144.0,NaN,-48.0,-44.0
강북구,-32.0,NaN,-21.0,-17.0
강서구,-160.0,NaN,-3.0,-21.0
관악구,-108.0,NaN,-44.0,-3.0
광진구,-92.0,NaN,23.0,-46.0
구로구,-87.0,NaN,-55.0,-7.0
금천구,-87.0,NaN,24.0,-5.0
노원구,-16.0,NaN,12.0,-11.0


In [21]:
#인덱싱 재처리
StoreCount= StoreCount.reset_index()
StoreCount.columns

# 지도영역 나누기
m = folium.Map([37.55784788416739, 127.0088620724302],
                   zoom_start = 11,
                   tiles = 'cartodbpositron'
)

folium.Choropleth(
    geo_data=seoul_geo,
    data=StoreCount,
    columns=['구','전통시장'],
    key_on='properties.name',
    fill_color = 'PuRd',
    fill_opacity=0.8,                    #투명도
    line_opacity=0.5,                    # 점선
    legend_name='점포수'             #이름  
    
).add_to(m)

m